# RAG chat

https://github.com/guidance-ai/guidance/blob/ae8830f69553fd658bf32e0fdb478cd9518bab9e/notebooks/art_of_prompt_design/rag.ipynb

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import stampy_nb


In [2]:
## secrets
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

# import warnings
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

## numeric, plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7.0, 4)

## utils
from pathlib import Path
from tqdm.auto import tqdm
import logging, os, re
import collections, functools, itertools

# logging
from loguru import logger
# logger.remove()
# logger.add(os.sys.stdout, level="ERROR", colorize=True, format="<level>{time} | {message}</level>")


/media/wassname/SGIronWolf/projects5/2024/stampy_nb/stampy_nb/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helpers

### Search and data

In [3]:
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")


BING_SEARCH_KEY = os.environ['BING_SEARCH_KEY']

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": BING_SEARCH_KEY}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        if "webPages" not in _bing_cache[params_key]:
            # no results
            logger.warning("No results found for %s", search_term)
            continue
        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["content"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results
    return [format_snippet(s) for s in search_results]

def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['content']} for x in results]

def format_snippet(s):
    return f"""url: {s['url']}\ntitle: {s['name']}\nextrat: {s['snippet']}""".strip()

q = "What are the main categories of mechinterp interventions?"
print(bing_search(q)[0])

{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0', 'name': '2: Types of intervention and their development - Medicine LibreTexts', 'url': 'https://med.libretexts.org/Bookshelves/Nursing/Field_Trials_of_Health_Interventions_-_A_Toolbox_(Smith_Morrow_and_Ross)/02%3A_Types_of_intervention_and_their_development', 'isFamilyFriendly': True, 'displayUrl': 'https://med.libretexts.org/Bookshelves/Nursing/Field_Trials_of_Health_<b>Interventions</b>_-_A...', 'snippet': '2.1: Introduction to <b>types</b> of <b>intervention</b> and their development This book is about the evaluation of the effectiveness of health-related <b>interventions</b>. We use the term ‘<b>intervention</b>’ to apply to any activity undertaken with the objective of improving human health by preventing disease, by curing or reducing the severity or duration of an existing disease, or by restoring function lost ...', 'deepLinks': [{'name': 'Types of Intervention', 'url': 'https://med.libretexts.org/Bookshelves/Nursing/F

In [4]:
# from datasets import load_dataset
# data = load_dataset('StampyAI/alignment-research-dataset', trust_remote_code=True)
# # TODO also search google and bing https://python.langchain.com/v0.2/docs/integrations/tools/search_tools/
# data

### Prompts

In [5]:
# https://github.com/StampyAI/stampy-chat/blob/main/api/src/stampy_chat/settings.py

SOURCE_PROMPT = (
    "You are a helpful assistant knowledgeable about AI Alignment and Safety. "
    "Please give a clear and coherent answer to the user's questions. (written after \"Q:\") "
    "using the following sources. Each source is labeled with a letter. Feel free to "
    "use the sources in any order, and try to use multiple sources in your answers.\n\n"
)
HISTORY_PROMPT = (
    "\n\n"
    "Before the question (\"Q: \"), there will be a history of previous questions and answers. "
    "These sources only apply to the last question. Any sources used in previous answers "
    "are invalid."
)
HISTORY_SUMMARIZE_PROMPT = (
    "You are a helpful assistant knowledgeable about AI Alignment and Safety. "
    "Please summarize the following chat history (written after \"H:\") in one "
    "sentence so as to put the current questions (written after \"Q:\") in context. "
    "Please keep things as terse as possible."
    "\nH:"
)

QUESTION_PROMPT = (
    "In your answer, please cite any claims you make back to each source "
    "using the format: [a], [b], etc. If you use multiple sources to make a claim "
    "cite all of them. For example: \"AGI is concerning [c, d, e].\"\n\n"
)
PROMPT_MODES = {
    'default': "",
    "concise": (
        "Answer very concisely, getting to the crux of the matter in as "
        "few words as possible. Limit your answer to 1-2 sentences.\n\n"
    ),
    "rookie": (
        "This user is new to the field of AI Alignment and Safety - don't "
        "assume they know any technical terms or jargon. Still give a complete answer "
        "without patronizing the user, but take any extra time needed to "
        "explain new concepts or to illustrate your answer with examples. "
        "Put extra effort into explaining the intuition behind concepts "
        "rather than just giving a formal definition.\n\n"
    ),
    "discord": (
        "Your answer will be used in a Discord channel, so please Answer concisely, getting to "
        "the crux of the matter in as few words as possible. Limit your answer to 1-2 paragraphs.\n\n"
    ),
}
DEFAULT_PROMPTS = {
    'context': SOURCE_PROMPT,
    'history': HISTORY_PROMPT,
    'history_summary': HISTORY_SUMMARIZE_PROMPT,
    'question': QUESTION_PROMPT,
    'modes': PROMPT_MODES,
}

In [6]:
import random
import guidance
from guidance import models, gen, select, substring, string, prefix_tree, regex, user, assistant, system
from guidance import silent, capture, Tool, one_or_more, any_char, commit_point

# llama2 = models.LlamaCpp("/home/marcotcr_google_com/work/models/llama-2-13b-chat.Q6_K.gguf", n_gpu_layers=-1, n_ctx=4096)
llama2 = models.OpenAI("gpt-3.5-turbo")

import sys
# logger.add(sys.stderr, format="{time} {level} {message}", level="INFO", colorize=True)
# logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
logger.info("test")

2024-06-22 09:41:17.104 | INFO     | __main__:<module>:12 - test


In [7]:
def format_snippets(snippets, start=1):
    ret = ''
    for i, s in enumerate(snippets, start=start):
        title = s['title']
        snippet = s['snippet']
        ret += f'[{i}] {title}\n'
        ret += f'{snippet}\n\n'
    return ret

@guidance
def search(lm, query):
    # Setting this for later use
    lm = lm.set('query', query)
    # This is where search actually gets called
    lm = lm.set('snippets', format_snippets(top_snippets(query)))
    lm += '\nObservation:\n' + lm['snippets']
    return lm

@guidance
def rephrase(lm, 
    query: str = "What is the main category of mechinterp interventions?"
):
    with silent():
        with system():
            lm += '''
            '''
        with user():
            lm += f"Please rewrite the following search query to use with common synonyms, keywords, and phrasing. The result should help find documents to answer this question: {query}"
        with assistant():
            lm += gen('q_rephrased')
    return lm


@guidance
def example_answer(lm,
                    query: str = "What is the main category of mechinterp interventions?"
                    ):
    with silent():
        with system():
            lm += '''
            '''
        with user():
            lm += f"Please draft a concrete and concise example answer that ties together all elements of the following question in a paragraph or less: {query}"
        with assistant():
            lm += gen('q_example_ans')    
    return lm

### Run

In [27]:
import re
import html
from IPython.core.display import HTML

def format_ans(ans, docs):
    pattern = r'\[\s*((?:\d+\s*(?:,\s*\d+\s*)*)?)\]'
    matches = re.findall(pattern, ans)
    # print(matches)
    text2 = ans + ''
    for m in matches:
        ns = m.strip(', ').split(', ')
        m2 = ''
        for n in ns:
            d = docs[int(n)]
            tooltip = html.escape(f"{d['name']}:\n\n{d['content']}").strip()
            m2 += f'<a href="{d["url"]}"><span title="{tooltip}">{n}</span></a>, '
        # print('m2', m2)
        text2 = text2.replace(m, m2)


    # now a list of refs
    refs = list(set(itertools.chain(*matches)))
    text2 += "\n\nReferences:\n\n"
    for r in refs:
        d = docs[int(r)]
        tooltip = html.escape(f"{d['name']}:\n\n{d['content']}")
        text2 += f'<a href="{d["url"]}">[{r}]</a>: {html.escape(d["name"])} - {html.escape(d["content"])}\n'

    text2 = text2.replace('\n', '</p>')
    # print('text', text)



    # print(text2)
    return HTML(text2)


# format_ans(ans, docs)

In [9]:
# Step 1 search
lm = llama2
q = "What are the main categories of mechinterp interventions?"


# Step 2 rephrase and example answers
docs = []
docs += bing_search(q)

q_rephrased = lm + rephrase(q)
q_rephrased = q_rephrased['q_rephrased'].strip('"\' ')
logger.info(f"Rephrased query: {q_rephrased}")
docs += bing_search(q_rephrased)

q_example_ans = lm + example_answer(q)
q_example_ans = q_example_ans['q_example_ans'].strip('"\' ')
logger.info(f"Example answer: {q_example_ans}")
docs += bing_search(q_example_ans)

random.shuffle(docs)

# Step 3 rerank
# TODO

# Step 3 summarize and generate
SOURCE_PROMPT
QUESTION_PROMPT




2024-06-22 09:41:20.465 | INFO     | __main__:<module>:17 - Example answer: Mechinterp interventions can be broadly categorized into three main groups: mechanical, interpretive, and integrative. Mechanical interventions involve physical adjustments or modifications to the environment or equipment to improve accessibility and usability. Interpretive interventions focus on providing explanations or translations to bridge communication gaps or clarify information. Integrative interventions aim to combine mechanical and interpretive strategies to create a seamless and inclusive experience for all individuals, ensuring that both physical and informational barriers are addressed effectively.


'In your answer, please cite any claims you make back to each source using the format: [a], [b], etc. If you use multiple sources to make a claim cite all of them. For example: "AGI is concerning [c, d, e]."\n\n'

In [13]:
# see make_prompt in stampy https://github.com/StampyAI/stampy-chat/blob/990c5dcad5721484c43f6297d84208614a5bf568/api/src/stampy_chat/chat.py#L245
lm = llama2

@guidance
def do_answers(lm, query, docs, history_summary=''):
    with system():
        lm += """You are a helpful assistant knowledgeable about AI Alignment and Safety. Please give a clear and coherent answer to the user\'s questions. (written after "Q:") using the following sources. Each source is labeled with a letter. Feel free to use the sources in any order, and try to use multiple sources in your answers"""
    with user():
        for i, d in enumerate(docs):
            lm += f"[{i}] {d['name']} {d['content']}\n\n"
        
        # lm += HISTORY_PROMPT
        lm += QUESTION_PROMPT + f'Q: {history_summary}: {query}'
    with assistant():
        lm += gen('ans')
    return lm

r = lm + do_answers(q, docs)
r

In [28]:
# now a nice html interface with tooltips and list of refs, need to replace numbers
ans = r['ans']
format_ans(ans, docs)